### Import Required Libraries and Set Up Environment Variables

In [28]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [29]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [30]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url= (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}')

In [31]:
# Create an empty list to store the reviews
reviews_list= []

# loop through pages 0-19
for page in range(0,20): 
    # create query with a page number
    # API results show 10 articles at a time
    page_query_url= f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews= requests.get(page_query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"no results")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [32]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [33]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df=pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [34]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"]=reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [35]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"]=reviews_df["keywords"].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [36]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles=reviews_df["title"].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [43]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [52]:
# Create an empty list to store the results
tmdb_movies_list= []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter= 1
requests_limit=0
pause_duration=1 
# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print(f"Reached {request_counter} requests. Sleeping for {pause_duration} seconds...")
        time.sleep(pause_duration)
    # Add 1 to the request counter
    request_counter += 1 
    
    # Perform a "GET" request for The Movie Database
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
    response = requests.get(url)
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
    try:
        movie_id = data['results'][0]['id']

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US"
        details_response = requests.get(details_url)
        details_data = details_response.json()
      
        # Execute "GET" request with url

        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in details_data.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in details_data.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in details_data.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            "title": title,
            "original_title": details_data["original_title"],
            "budget": details_data["budget"],
            "original_language": details_data["original_language"],
            "homepage": details_data["homepage"],
            "overview": details_data["overview"],
            "popularity": details_data["popularity"],
            "runtime": details_data["runtime"],
            "revenue": details_data["revenue"],
            "release_date": details_data["release_date"],
            "vote_average": details_data["vote_average"],
            "vote_count": details_data["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }

        tmdb_movies_list.append(movie_data)
       
        # Print out the title that was found
        print(f"Found movie: {title}")

    except (IndexError, KeyError):
       print(f"Movie not found: {title}")

    print("All movie details processed.")

Found movie: The Attachment Diaries
All movie details processed.
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
All movie details processed.
Found movie: You Can Live Forever
All movie details processed.
Found movie: A Tourist’s Guide to Love
All movie details processed.
Found movie: Other People’s Children
All movie details processed.
Found movie: One True Loves
All movie details processed.
Found movie: The Lost Weekend: A Love Story
All movie details processed.
Found movie: A Thousand and One
All movie details processed.
Found movie: Your Place or Mine
All movie details processed.
Found movie: Love in the Time of Fentanyl
All movie details processed.
Found movie: Pamela, a Love Story
All movie details processed.
Found movie: In From the Side
All movie details processed.
Found movie: After Love
All movie details processed.
Found movie: Alcarràs
All movie details processed.
Found movie: Nelly & Nadine
All movie details processed.
Found movie: Lady Chatterley’s Lover
All

In [59]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview = tmdb_movies_list[:5]
preview_json = json.dumps(preview, indent=4)

print(preview_json)

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.975,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [63]:
# Convert the results to a DataFrame
tmdb_df= pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.975,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,15055,2023-03-24,6.600,37,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,0,2023-04-21,6.287,162,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.071,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,17.104,100,37820,2023-04-07,6.500,75,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",20.906,130,0,2017-01-11,5.600,193,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,8.736,83,0,2017-01-14,6.022,114,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,7.487,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",42.073,111,413844,2017-03-10,6.164,247,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [89]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df, reviews_df, on='title', how='outer')
print("Merged DataFrame created.")
merged_df

Merged DataFrame created.


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,69: The Saga of Danny Hernandez,69: The Saga of Danny Hernandez,0.0,en,https://www.hulu.com/movie/69-the-saga-of-dann...,"Part investigative documentary, part real-life...",10.545,101.0,0.0,2021-04-29,...,‘69: The Saga of Danny Hernandez’ Review: A So...,None,None,"Loved in Hip-Hop, but Loathed in the End",None,None,None,By Calum Marsh,"[{'firstname': 'Calum', 'middlename': None, 'l...",None
1,A Journal for Jordan,A Journal for Jordan,25000000.0,en,https://www.sonypictures.com/movies/ajournalfo...,Based on the true story of First Sergeant Char...,12.560,131.0,6700000.0,2021-12-22,...,‘A Journal for Jordan’ Review: Reflections on ...,None,None,Reflections on Love That’s Built and Lost,None,None,None,By Lisa Kennedy,"[{'firstname': 'Lisa', 'middlename': None, 'la...",None
2,A Love Song,A Love Song,0.0,en,,"Two childhood sweethearts, now both widowed, s...",23.717,81.0,278562.0,2022-07-29,...,‘A Love Song’ Review: When Moving Forward Mean...,Critic’s Pick,None,The Romantic Ache That Never Truly Dies,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,A Secret Love,A Secret Love,0.0,en,https://www.netflix.com/title/80209024,"Amid shifting times, two women kept their deca...",15.492,83.0,0.0,2020-04-29,...,‘A Secret Love’ Review: A Lesbian Couple’s End...,None,None,Making a Haven of Their Own,None,None,None,By Natalia Winkelman,"[{'firstname': 'Natalia', 'middlename': None, ...",None
4,A Thousand and One,A Thousand and One,0.0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.785,116.0,3395595.0,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,You Go to My Head,You Go To My Head,0.0,fr,http://yougotomyhead.com/,Following a mysterious car accident in the des...,6.262,116.0,0.0,2017-10-26,...,‘You Go to My Head’ Review: Untrue Love,Critic’s Pick,None,Designing a Love Story,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
196,Young Hearts,Young Hearts,0.0,nl,https://www.polar-bear.tv/young-hearts,Fourteen-year-old Elias increasingly feels lik...,18.527,97.0,0.0,2024-10-10,...,‘Young Hearts’ Review: Movie Love by Algorithm,None,None,Young Hearts,None,None,None,By Kristen Yoonsoo Kim,"[{'firstname': 'Kristen', 'middlename': 'Yoons...",None
197,Your Name Engraved Herein,刻在你心底的名字,0.0,zh,,"In 1987, as martial law ends in Taiwan, Jia-ha...",19.037,114.0,0.0,2020-09-30,...,‘Your Name Engraved Herein’ Review: When Love ...,None,None,Your Name Engraved Herein,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
198,Your Place or Mine,Your Place or Mine,0.0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,14.319,109.0,0.0,2023-02-10,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [68]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(character, "")

# Display the fixed DataFrame
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,69: The Saga of Danny Hernandez,69: The Saga of Danny Hernandez,0.0,en,https://www.hulu.com/movie/69-the-saga-of-dann...,"Part investigative documentary, part real-life...",10.545,101.0,0.0,2021-04-29,...,‘69: The Saga of Danny Hernandez’ Review: A So...,None,None,"Loved in Hip-Hop, but Loathed in the End",None,None,None,By Calum Marsh,"[{'firstname': 'Calum', 'middlename': None, 'l...",None
1,A Journal for Jordan,A Journal for Jordan,25000000.0,en,https://www.sonypictures.com/movies/ajournalfo...,Based on the true story of First Sergeant Char...,12.560,131.0,6700000.0,2021-12-22,...,‘A Journal for Jordan’ Review: Reflections on ...,None,None,Reflections on Love That’s Built and Lost,None,None,None,By Lisa Kennedy,"[{'firstname': 'Lisa', 'middlename': None, 'la...",None
2,A Love Song,A Love Song,0.0,en,,"Two childhood sweethearts, now both widowed, s...",23.717,81.0,278562.0,2022-07-29,...,‘A Love Song’ Review: When Moving Forward Mean...,Critic’s Pick,None,The Romantic Ache That Never Truly Dies,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,A Secret Love,A Secret Love,0.0,en,https://www.netflix.com/title/80209024,"Amid shifting times, two women kept their deca...",15.492,83.0,0.0,2020-04-29,...,‘A Secret Love’ Review: A Lesbian Couple’s End...,None,None,Making a Haven of Their Own,None,None,None,By Natalia Winkelman,"[{'firstname': 'Natalia', 'middlename': None, ...",None
4,A Thousand and One,A Thousand and One,0.0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.785,116.0,3395595.0,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,You Go to My Head,You Go To My Head,0.0,fr,http://yougotomyhead.com/,Following a mysterious car accident in the des...,6.262,116.0,0.0,2017-10-26,...,‘You Go to My Head’ Review: Untrue Love,Critic’s Pick,None,Designing a Love Story,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
196,Young Hearts,Young Hearts,0.0,nl,https://www.polar-bear.tv/young-hearts,Fourteen-year-old Elias increasingly feels lik...,18.527,97.0,0.0,2024-10-10,...,‘Young Hearts’ Review: Movie Love by Algorithm,None,None,Young Hearts,None,None,None,By Kristen Yoonsoo Kim,"[{'firstname': 'Kristen', 'middlename': 'Yoons...",None
197,Your Name Engraved Herein,刻在你心底的名字,0.0,zh,,"In 1987, as martial law ends in Taiwan, Jia-ha...",19.037,114.0,0.0,2020-09-30,...,‘Your Name Engraved Herein’ Review: When Love ...,None,None,Your Name Engraved Herein,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
198,Your Place or Mine,Your Place or Mine,0.0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,14.319,109.0,0.0,2023-02-10,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [75]:
# Drop "byline.person" column
merged_df.drop('byline.person', axis=1, inplace=True)

In [76]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,69: The Saga of Danny Hernandez,69: The Saga of Danny Hernandez,0.0,en,https://www.hulu.com/movie/69-the-saga-of-dann...,"Part investigative documentary, part real-life...",10.545,101.0,0.0,2021-04-29,...,363,‘69: The Saga of Danny Hernandez’ Review: A So...,None,None,"Loved in Hip-Hop, but Loathed in the End",None,None,None,By Calum Marsh,None
1,A Journal for Jordan,A Journal for Jordan,25000000.0,en,https://www.sonypictures.com/movies/ajournalfo...,Based on the true story of First Sergeant Char...,12.560,131.0,6700000.0,2021-12-22,...,505,‘A Journal for Jordan’ Review: Reflections on ...,None,None,Reflections on Love That’s Built and Lost,None,None,None,By Lisa Kennedy,None
2,A Love Song,A Love Song,0.0,en,,"Two childhood sweethearts, now both widowed, s...",23.717,81.0,278562.0,2022-07-29,...,395,‘A Love Song’ Review: When Moving Forward Mean...,Critic’s Pick,None,The Romantic Ache That Never Truly Dies,None,None,None,By Jeannette Catsoulis,None
3,A Secret Love,A Secret Love,0.0,en,https://www.netflix.com/title/80209024,"Amid shifting times, two women kept their deca...",15.492,83.0,0.0,2020-04-29,...,309,‘A Secret Love’ Review: A Lesbian Couple’s End...,None,None,Making a Haven of Their Own,None,None,None,By Natalia Winkelman,None
4,A Thousand and One,A Thousand and One,0.0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.785,116.0,3395595.0,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,You Go to My Head,You Go To My Head,0.0,fr,http://yougotomyhead.com/,Following a mysterious car accident in the des...,6.262,116.0,0.0,2017-10-26,...,292,‘You Go to My Head’ Review: Untrue Love,Critic’s Pick,None,Designing a Love Story,None,None,None,By Jeannette Catsoulis,None
196,Young Hearts,Young Hearts,0.0,nl,https://www.polar-bear.tv/young-hearts,Fourteen-year-old Elias increasingly feels lik...,18.527,97.0,0.0,2024-10-10,...,324,‘Young Hearts’ Review: Movie Love by Algorithm,None,None,Young Hearts,None,None,None,By Kristen Yoonsoo Kim,None
197,Your Name Engraved Herein,刻在你心底的名字,0.0,zh,,"In 1987, as martial law ends in Taiwan, Jia-ha...",19.037,114.0,0.0,2020-09-30,...,295,‘Your Name Engraved Herein’ Review: When Love ...,None,None,Your Name Engraved Herein,None,None,None,By Teo Bugbee,None
198,Your Place or Mine,Your Place or Mine,0.0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,14.319,109.0,0.0,2023-02-10,...,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None


In [90]:
# Export data to CSV without the index

merged_df.to_csv('merged_movies.csv', index=False)